In [ ]:
### Load Dependencies ###

import teradata
import pandas
import os
import config

In [ ]:
### Framework for executing Teradata queries

def exec_query(query, session, verbose = True, dryrun = False, ignore_db_error = False):

    if dryrun: print('--- DRY RUN ONLY - NO EXECUTION ---')
    print(str(pandas.datetime.now()))
    print(query)

    try:
        if not dryrun: session.execute(query)
        print(str(pandas.datetime.now()))
        print('DONE.')
        return None

    except teradata.api.DatabaseError as err:
        print(str(pandas.datetime.now()))
        print("DATABASE ERROR: ", err)
        
        if not ignore_db_error: raise ValueError("DATABASE ERROR: ", err)
        if ignore_db_error: print("Ignoring Database Error.")

        return None
    
    except:
        print(str(pandas.datetime.now()))
        print("FATAL ERROR")
        raise
        return None

In [ ]:
### Setup and open Teradata connection ###

os.system('echo -n "$TERADATA_LOGON_PASSWORD" | kinit "$TERADATA_LOGON_USER"');

In [ ]:
udaExec = teradata.UdaExec(appName="python", version="1.0", logConsole=False,
                          odbcLibPath="/opt/app/ttu1510/teradata/client/15.10/odbc_64/lib/libodbc.so")
session = udaExec.connect(method="odbc", system="edwprod.dw.medcity.net", driver="Teradata")

### Create Array of Indicies

In [ ]:
query = "SELECT * FROM " + config.INDEX_TABLE + ";"

In [ ]:
index = pandas.read_sql(query,session)

In [ ]:
index.sort_values("INDEX_SEQ").reset_index(drop=True)

In [ ]:
idx = index.sort_values("INDEX_SEQ")[['KEYFRAME_SEQ','INTRAFRAME_SEQ']].astype(int).values.tolist()

In [ ]:
base_idx = idx[0]
intra_idx = idx[1:7]

In [ ]:
base_idx

In [ ]:
intra_idx

### Checks

In [ ]:
print(config.PLAYBACK_FRAME)
print(config.PLAYBACK_APPEND)
print(config.PLAYBACK_REMOVE)
print(config.PLAYBACK_TEMP)

### Initialize Frame Constructions

In [ ]:
# Reset Output Table

query = "DROP TABLE " + config.PLAYBACK_FRAME + ";"

exec_query(query, session, ignore_db_error=True)

In [ ]:
# Create Output Table

queries = [
    """
    CREATE TABLE  """ + config.PLAYBACK_FRAME + """ AS (
    SELECT * FROM """ + config.KEYFRAME_TABLE + """
    WHERE KEYFRAME_SEQ = """ + str(base_idx[0]) + """
    ) WITH DATA PRIMARY INDEX (""" + config.PRIMARY_KEY + """);
    """,
    "ALTER TABLE " + config.PLAYBACK_FRAME + " DROP FRAME_TIMESTAMP;",
    "ALTER TABLE " + config.PLAYBACK_FRAME + " DROP KEYFRAME_SEQ;",
    "ALTER TABLE " + config.PLAYBACK_FRAME + " DROP INTRAFRAME_SEQ;",
    "ALTER TABLE " + config.PLAYBACK_FRAME + " DROP HASH_VALUE;"
    ]

for query in queries:
    exec_query(query, session)


### Construct Frames

In [ ]:
for i in intra_idx:
    queries = [
        """
        --Create list of records that will be altered
        CREATE TABLE """ + config.PLAYBACK_REMOVE + """ AS (
        SELECT """ + config.PRIMARY_KEY + """ AS """ + config.PRIMARY_KEY + """_REMOVE,
        1 AS FLAG_REMOVE
        FROM """ + config.INTRAFRAME_TABLE + """
        WHERE KEYFRAME_SEQ = """ + str(i[0]) + """
        AND INTRAFRAME_SEQ = """ + str(i[1]) + """
        ) WITH DATA PRIMARY INDEX (""" + config.PRIMARY_KEY + """_REMOVE);
        """,

        """
        --Create list of records that will be replaced or added
        CREATE TABLE """ + config.PLAYBACK_APPEND + """ AS (
        SELECT *
        FROM """ + config.INTRAFRAME_TABLE + """
        WHERE KEYFRAME_SEQ = """ + str(i[0]) + """
        AND INTRAFRAME_SEQ = """ + str(i[1]) + """
        AND MEMO in ('CHANGE', 'ADD')
        ) WITH DATA PRIMARY INDEX (""" + config.PRIMARY_KEY + """);
        """,
        
        "ALTER TABLE "  + config.PLAYBACK_APPEND + "  DROP FRAME_TIMESTAMP;",
        "ALTER TABLE "  + config.PLAYBACK_APPEND + "  DROP KEYFRAME_SEQ;",
        "ALTER TABLE "  + config.PLAYBACK_APPEND + "  DROP INTRAFRAME_SEQ;",
        "ALTER TABLE "  + config.PLAYBACK_APPEND + "  DROP HASH_VALUE;",
        "ALTER TABLE "  + config.PLAYBACK_APPEND + "  DROP MEMO;",

        """
        --Remove Records that will be changed or dropped

        CREATE VOLATILE TABLE """ + config.PLAYBACK_TEMP + """,
        NO LOG AS (
        SELECT * FROM """ + config.PLAYBACK_FRAME + """ base
        LEFT JOIN """ + config.PLAYBACK_REMOVE + """ rmv
        ON base.""" + config.PRIMARY_KEY + """ = rmv.""" + config.PRIMARY_KEY + """_REMOVE
        ) WITH DATA
        PRIMARY INDEX (""" + config.PRIMARY_KEY + """)
        ON COMMIT PRESERVE ROWS;
        """,
        "DELETE FROM " + config.PLAYBACK_TEMP + " WHERE FLAG_REMOVE = 1;",

        
        "DROP TABLE " + config.PLAYBACK_FRAME + ";",
        "CREATE TABLE " + config.PLAYBACK_FRAME + """ AS 
        """ + config.PLAYBACK_TEMP + " WITH DATA AND STATS PRIMARY INDEX (" + config.PRIMARY_KEY + ");",
        "ALTER TABLE " + config.PLAYBACK_FRAME + " DROP " + config.PRIMARY_KEY + "_REMOVE;",
        "ALTER TABLE " + config.PLAYBACK_FRAME + " DROP FLAG_REMOVE;",
        "DROP TABLE " + config.PLAYBACK_TEMP + ";",

        """
        --Add Records that will have been changed or added

        INSERT INTO """ + config.PLAYBACK_FRAME + """
        SELECT * FROM """ + config.PLAYBACK_APPEND + """;
        """,
        
        """
        --Clear out temp tables

        DROP TABLE """ + config.PLAYBACK_REMOVE + """;
        """,
        """
        DROP TABLE """ + config.PLAYBACK_APPEND + """;
        """
    ]
    for query in queries:
        exec_query(query, session)

### Close Teradata Connection

In [ ]:
### Close Session ###
session.close()

print(str(pandas.datetime.now()))
print("Session Closed.")